In [1]:
# 1.문서 내용을 읽어옴. / 2. 문서를 쪼갬.
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter( # 긴 텍스트를 LLM이 처리하기 좋게끔 조각으로 쪼개주는 함수
    chunk_size=1500, # 한 조각의 최대 문자 수
    chunk_overlap=200, # 이전 조각과 겹치는 문자 수 (문맥 손실 방지)
)

loader = Docx2txtLoader("./tax.docx") # 해당 문서를 읽기 위한 로더 객체를 생성함.
document_list = loader.load_and_split(text_splitter=text_splitter) 

In [2]:
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings

load_dotenv()

embedding = OpenAIEmbeddings(model='text-embedding-3-large')

In [ ]:
# 3.임베딩하여 데이터베이스에 저장함.
import os
from pinecone import Pinecone # 클라우드 기반 데이터베이스 SaaS
from langchain_pinecone import PineconeVectorStore

pinecone_api_key = os.environ.get("PINECONE_API_KEY")
pc = Pinecone(api_key=pinecone_api_key) # Pinecone 서버와 통신하기 위한 클라이언트 객체를 생성함.

index_name='tax-index'
database = PineconeVectorStore.from_documents( 
    document_list, 
    embedding, # 주어진 문서에 해당 임베딩을 적용하여 벡터로 변환함.
    index_name=index_name # 데이터베이스에 해당 인덱스명으로 벡터를 저장함.
)


/Users/gimseongjin/Inf_LangGraph/stage2/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
query = '연봉 5천만원인 직장인의 소득세는 얼마인가요?'

# 4.질문에 대한 유사도를 검색함.
retrieved_docs = database.similarity_search(query, k=3) # 질문(쿼리)을 임베딩한 후, 데이터베이스에 저장된 모든 문서 조각들과 유사도를 비교한 후, 유사도가 가장 높은 k개만 반환함.

In [5]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='gpt-4o')

In [6]:
# 5.유사도 검색 문서와 질문을 LLM에게 전달함.

prompt = f"""[identity]
    - 당신은 최고의 한국 소득세 전문가 입니다
    - [Context]를 참고해서 사용자의 질문에 답변해주세요
    
    [Context]
    {retrieved_docs}
    
    Question: {query}
"""

In [7]:
ai_message = llm.invoke(prompt)

In [8]:
ai_message.content

'연봉 5천만 원인 직장인의 소득세를 계산하기 위해서는 몇 가지 요소를 고려해야 합니다. 우선, 종합소득세 과세표준을 계산해야 하며, 기본적인 소득공제를 적용한 후 해당 과세표준에 대한 세율을 적용하여 소득세를 산출합니다. \n\n한국의 소득세율은 누진세 구조로 되어 있어서, 소득이 많을수록 높은 세율이 적용됩니다. 2023년 기준으로, 종합소득 총액에 따른 세율은 다음과 같습니다.\n\n1. 1,200만 원 이하: 6%\n2. 1,200만 원 초과 4,600만 원 이하: 15%\n3. 4,600만 원 초과 8,800만 원 이하: 24%\n4. 8,800만 원 초과 1억 5천만 원 이하: 35%\n5. 1억 5천만 원 초과 3억 원 이하: 38%\n6. 3억 원 초과 5억 원 이하: 40%\n7. 5억 원 초과 10억 원 이하: 42%\n8. 10억 원 초과: 45%\n\n연봉 5천만 원인 경우, 과세표준은 대략 5,000만 원에서 근로소득공제와 기타 공제를 뺀 금액입니다. 여기에서는 정확한 공제 금액을 알 수 없으므로 일반적인 근로소득 공제를 적용하여 설명하겠습니다.\n\n1. 근로소득공제(대략): 5000만 원의 경우 약 1110만 원의 근로소득공제가 됩니다.\n2. 과세표준: 약 3890만 원 (5천만 원 - 1110만 원)\n\n해당 금액에 대한 세율은 15% 구간과 6% 구간이 적용됩니다.\n\n- 1,200만 원 이하: 72만 원 (1,200만 원 × 6%)\n- 초과 2,690만 원 (3,890만 원 - 1,200만 원): 403.5만 원 (2,690만 원 × 15%)\n\n따라서, 총 소득세는 약 475.5만 원입니다. 추가로, 주민세 등 다른 세금도 고려해야 하기 때문에 실제 납부 금액은 약간 다를 수 있습니다. 더 정확한 계산을 위해서는 개인의 공제항목 및 세액공제 항목을 구체적으로 반영하여 계산해야 합니다.'

### RetrievalQA 활용

In [9]:
from langchain_classic import hub

prompt = hub.pull("rlm/rag-prompt") # RAG용 표준 프롬프트를 LangChain Hub에서 가져옴. ("작성자/프롬프트명")

In [10]:
# 4.질문에 대한 유사도를 검색함. / 5.유사도 검색 문서와 질문을 LLM에게 전달함.
from langchain_classic.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type( # 벡터 검색, 프롬프트, LLM을 하나로 묶은 RAG 체인을 생성함. 
    retriever = database.as_retriever(search_kwargs={"k": 3}), # 데이터베이스를 리트리버 인터페이스로 변환함. 
    chain_type_kwargs = {"prompt": prompt}, # 사용자 지정 프롬프트로 교체함.
    llm = llm  
)

ai_message = qa_chain({"query": query})

/var/folders/5q/h0j_qc4x00x4kwkjdh13l3q80000gn/T/ipykernel_18780/429544826.py:10: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain-classic 0.1.0 and will be removed in 1.0. Use `invoke` instead.
  ai_message = qa_chain({"query": query})


In [11]:
ai_message['result']

'죄송합니다. 제공된 문맥에 따르면 연봉 5천만 원인 직장인의 소득세 금액에 대한 정보는 포함되어 있지 않습니다. 소득세는 일반적으로 여러 요인에 따라 달라질 수 있으므로, 구체적인 계산은 별도의 세금 계산기나 세무전문가의 조언을 통해 확인하는 것이 좋습니다.'

### 키워드 사전 활용

In [20]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

query = '연봉 5천만원인 직장인의 소득세는 얼마인가요?'

dictionary = ["사람을 나타내는 표현 -> 거주자"]

prompt = ChatPromptTemplate.from_template(f"""
        우리의 사전을 참고해서 사용자의 질문을 변경해주세요.
        만약, 변경하지 않아도 될것 같다면, 그대로 사용자의 질문을 사용해주세요.
        사전: {dictionary}
        
        질문: {{question}}     
""")

dictionary_chain = prompt | llm | StrOutputParser()

In [21]:
tax_chain = {"query": dictionary_chain} | qa_chain

In [22]:
ai_response = tax_chain.invoke({"question": query})

In [25]:
ai_response

{'query': '질문: 연봉 5천만원인 거주자의 소득세는 얼마인가요?',
 'result': '연봉 5천만원인 거주자의 소득세는 624만원입니다. 이는 제시된 세율 표에 따라 5,000만원까지의 소득에 적용되는 세율을 기준으로 계산된 금액입니다.'}